# Marketstack: notebook to generate Ticker CSVs from MS API

- ingest the close values for a timeframe
- convert the relevant part of the JSON that comes back into a dataframe
- convert the date to just yyyy-mm-dd and make that value the index of the dataframe
- 3250 - 6 = 3244 CSV files generated (from 3261 unique input tickers)
- 17 tickers unaccounted for - captured these in batch_ticker_retry_aug9.csv and rerunning
- single source of truth for generated ticker CSVs: https://drive.google.com/drive/folders/15AsjQAWzrrTko5OuK7idntgKcX0181sf?usp=sharing


In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from io import StringIO
import yaml
from datetime import date
import requests
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
#import pandas_datareader as pdr
from pandas_datareader.data import DataReader
# import yahoo_fin.stock_info as si

# For time stamps
from datetime import datetime

# for LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
from keras.models import Model
# from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
# access datasets from quandl.com - need to pip install Quandl to use
import quandl
config_file = 'batch_close_load_config.yml'

Using TensorFlow backend.


In [2]:
# load config file
current_path = os.getcwd()
print("current directory is: "+current_path)

path_to_yaml = os.path.join(current_path, config_file)
print("path_to_yaml "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file')

current directory is: C:\personal\karma_stocks_2021\stock_investigation\notebooks
path_to_yaml C:\personal\karma_stocks_2021\stock_investigation\notebooks\batch_close_load_config.yml


In [3]:
# load config parms
access_key = config['general']['marketstack_key']
# file containing list of tickers to batch load
batch_ticker_list_file = config['files']['batch_ticker_list_file']
from_date = config['general']['master_start']
to_date = config['general']['master_end']

In [4]:
# get raw data from marketstack
def get_close_data(symbol='AAPL.US', api_token='OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX', from_date = '2021-07-01', to_date = '2021-07-08'):
    session = requests.Session()
    print("symbol is ",symbol)
    print("from_date ",from_date)
    print("to_date ", to_date)
    # base_url = 'http://api.marketstack.com/v1/tickers/'+symbol+'/eod'
    base_url = 'http://api.marketstack.com/v1/eod'
    #url = 'https://eodhistoricaldata.com/api/div/%s' % symbol
    params = {'access_key': api_token,'symbols':symbol,'limit':10000,'date_from':from_date,'date_to':to_date}
    r = session.get(base_url, params=params)
    
    ticker_dict = json.loads(r.text)["data"]
    # print("ticker_dict is ", ticker_dict)
    if r.status_code == requests.codes.ok:
        #print("status OK ")
        df = pd.DataFrame.from_dict(ticker_dict, orient='columns')
        #df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[0], index_col=0, engine='python')
        return(True, df)
    else:
        print("status code",str(r.status_code))
        print("reason code",str(r.reason))
        return(False,"null")
        #raise Exception(r.status_code, r.reason, url)

In [5]:
# clean up dataframe returned by marketstack
def ms_df_cleanup(df):
    # keep just the yyyy-mm-dd portion of date column
    df['date'] = df['date'].str[0:10]
    # rename the column to be consistent
    df.rename(columns = {'date': 'Date','close': 'Close'}, inplace = True)
    # set Date to be the index
    df.set_index('Date', inplace = True)
    return(df)

In [6]:
def get_directory_path(directory):
    '''get the fully qualified path for a peer directory of the directory where this notebook is run'''
    rawpath = os.getcwd()
    # "directory" is in a directory is a sibling to the directory containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', directory))
    return(path)

In [7]:
# main loop
batch_file_name = os.path.join(get_directory_path('data'), batch_ticker_list_file)
print("file name is", batch_file_name)
print("from date ",from_date)
print("to date ", to_date)
df_batch = pd.read_csv(batch_file_name)
i = 0
i_max = 10
for ticker in df_batch['Ticker']:
#    if i > i_max:
#        break
#    i = i+1
#    print(" i is ",i)
    print("ticker is ",ticker)
    result, df_out = get_close_data(ticker,access_key,from_date, to_date)
    df = ms_df_cleanup(df_out)
    ticker_file_name = ticker+'_'+from_date+'_'+to_date+'.csv'
    ticker_batch_file = os.path.join(get_directory_path('static_load'), ticker_file_name)
    df.to_csv(ticker_batch_file)
print("through main loop")
'''
result, df_out = get_close_data('aapl',access_key,from_date, to_date)
df = ms_df_cleanup(df_out)
'''

file name is C:\personal\karma_stocks_2021\stock_investigation\data\batch_ticker_sector_update_aug25.csv
from date  1991-01-01
to date  2021-08-27
ticker is  IDU
symbol is  IDU
from_date  1991-01-01
to_date  2021-08-27
ticker is  IWV
symbol is  IWV
from_date  1991-01-01
to_date  2021-08-27
ticker is  IYE
symbol is  IYE
from_date  1991-01-01
to_date  2021-08-27
ticker is  VDE
symbol is  VDE
from_date  1991-01-01
to_date  2021-08-27
ticker is  VHT
symbol is  VHT
from_date  1991-01-01
to_date  2021-08-27
ticker is  VIS
symbol is  VIS
from_date  1991-01-01
to_date  2021-08-27
ticker is  VNQ
symbol is  VNQ
from_date  1991-01-01
to_date  2021-08-27
ticker is  VOO
symbol is  VOO
from_date  1991-01-01
to_date  2021-08-27
ticker is  VPU
symbol is  VPU
from_date  1991-01-01
to_date  2021-08-27
ticker is  VTI
symbol is  VTI
from_date  1991-01-01
to_date  2021-08-27
ticker is  XLC
symbol is  XLC
from_date  1991-01-01
to_date  2021-08-27
ticker is  XLY
symbol is  XLY
from_date  1991-01-01
to_date  

"\nresult, df_out = get_close_data('aapl',access_key,from_date, to_date)\ndf = ms_df_cleanup(df_out)\n"